In [ ]:
# Based on city, get some general information required by TARGET
def get_city_info(city):

  if city == 'Timbuktu':
    city_info = {
        'lat': 16.774166,  # Use in UTCI calculation
        'domainDim': 217, # Number of FIDs in LC file?
        # 'latEdge': , # not used, ignore
        # 'lonEdge': , # not used, ignore?
        # 'latResolution': , # not used, ignore
        # 'lonResolution': , # not used, ignore
        'date1a': '2013,5,20,0',
        'date1': '2013,5,20,0',
        'date2': '2013,5,26,23',  # Bug in code, last timestamp not modelled?
    }
  #TODO!!
  elif city == 'Davao':
        city_info = {
        'lat': 7.088082,  # Use in UTCI calculation
        'domainDim': 674, # Number of FIDs in LC file
        # 'latEdge': , # not used, ignore
        # 'lonEdge': , # not used, ignore
        # 'latResolution': , # not used, ignore
        # 'lonResolution': , # not used, ignore
        'date1a': '2018,4,24,0',
        'date1': '2018,4,24,0',
        'date2': '2018,4,30,0', # Bug in code, last timestamp not modelled?
    }

  return city_info


In [ ]:
def make_interactive_map_roi(MODEL_INPUT_FOLDER):

    import os
    import geopandas as gpd
    import folium
    import matplotlib.cm as mpl_cm
    import matplotlib.colors as mpl_colors
    import branca.colormap as cm
    import numpy as np
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    import pandas as pd

    # Read the hexagons
    gdf_hex = gpd.read_file(
        os.path.join(os.path.dirname(MODEL_INPUT_FOLDER), "roi_hex_utm.geojson")
    )

    # Read the fractions
    df_fractions = pd.read_csv(
        os.path.join(MODEL_INPUT_FOLDER, "LC", "lc_target.csv"),
        index_col="FID"
    )

    # Merge with hexagons
    gdf_merged = gdf_hex.merge(df_fractions, left_on='hex_id', right_index=True, how='left')
    gdf_merged.set_index('hex_id', inplace=True)

    gdf_merged["hex_id"] = gdf_merged.index  # restore as column

    # Ensure WGS84
    gdf_merged = gdf_merged.to_crs(epsg=4326)

    # Define variable groups
    surface_vars = ['road', 'conc', 'roof', 'Veg', 'dry', 'irr', 'watr', 'H', 'W']

    pretty_names = {
        'road': 'Fraction of asphalted roads',
        'conc': 'Fraction of bare soil/sand (Timbuktu) / concrete (Davao)',
        'roof': 'Fraction of buildings',
        'Veg': 'Fraction of trees',
        'dry': 'Fraction of grass',
        'irr': 'Fraction of irrigated grass',
        'watr': 'Fraction of water',
        'H': 'Average building height (m)',
        'W': 'Average street width (m)',
    }


    # Define variable-specific colormaps
    var_cmap_dict = {
        # Surface fractions
        'road': mpl_cm.Greys,
        'conc': mpl_cm.Oranges,
        'roof': mpl_cm.Reds,
        'Veg': mpl_cm.Greens,
        'dry': mpl_cm.Greens,
        'irr': mpl_cm.Blues,
        'watr': mpl_cm.PuBu,
        'H': mpl_cm.inferno,
        'W': mpl_cm.plasma,
    }

    # Create dropdown menu
    dropdown = widgets.Dropdown(
        options=[(label, var) for var, label in pretty_names.items()],
        value='roof',
        description='Variable:',
        layout=widgets.Layout(width='50%')
    )

    # Output widget for the map
    out = widgets.Output()

    # Map update function
    def update_map(change=None):
        variable = dropdown.value
        cmap = var_cmap_dict.get(variable, mpl_cm.viridis)

        vmin = gdf_merged[variable].min()
        vmax = gdf_merged[variable].max()

        scalar_map = cm.StepColormap(
            colors=[mpl_colors.to_hex(cmap(i)) for i in np.linspace(0, 1, 256)],
            vmin=vmin, vmax=vmax,
            caption=pretty_names[variable]
        )

        center = [
            gdf_merged.geometry.centroid.y.mean(),
            gdf_merged.geometry.centroid.x.mean()
        ]

        m = folium.Map(
            location=center,
            zoom_start=14,
            # tiles='cartodbpositron',
            tiles="https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}",
            attr='Google',
            name='Google Satellite',
            width='90%',
            height='750px'
        )

        def style_function(feature):
            val = feature['properties'].get(variable)
            return {
                'fillOpacity': 0.7,
                'weight': 0.2,
                'color': 'black',
                'fillColor': scalar_map(val) if val is not None else '#999999'
            }

        folium.GeoJson(
            gdf_merged.to_json(),
            style_function=style_function,
            tooltip=folium.GeoJsonTooltip(
                fields=["hex_id", variable],
                aliases=["Hex ID: ", pretty_names[variable] + ": "],
                localize=True
            )
        ).add_to(m)

        scalar_map.add_to(m)

        with out:
            clear_output(wait=True)
            display(m)

    # Connect dropdown to map update
    dropdown.observe(update_map, names='value')

    # Initial display
    update_map()

    # Display dropdown and map
    display(widgets.VBox([dropdown, out]))

In [ ]:
def make_static_map_roi(MODEL_INPUT_FOLDER, figname):

    import os
    import geopandas as gpd
    import folium
    import matplotlib.cm as mpl_cm
    import matplotlib.colors as mpl_colors
    import branca.colormap as cm
    import numpy as np
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    import pandas as pd
    import matplotlib.pyplot as plt

    # Read the hexagons
    gdf_hex = gpd.read_file(
        os.path.join(os.path.dirname(MODEL_INPUT_FOLDER), "roi_hex_utm.geojson")
    )

    # Read the fractions
    df_fractions = pd.read_csv(
        os.path.join(MODEL_INPUT_FOLDER, "LC", "lc_target.csv"),
        index_col="FID"
    )

    # Merge with hexagons
    gdf_merged = gdf_hex.merge(df_fractions, left_on='hex_id', right_index=True, how='left')
    gdf_merged.set_index('hex_id', inplace=True)

    # Ensure WGS84
    gdf_merged = gdf_merged.to_crs(epsg=4326)

    # Define variable groups
    surface_vars = ['road', 'conc', 'roof', 'Veg', 'dry', 'irr', 'watr', 'H', 'W']

    pretty_names = {
        'road': 'Fraction of asphalted roads',
        'conc': 'Fraction of bare soil/sand',
        'roof': 'Fraction of buildings',
        'Veg': 'Fraction of trees',
        'dry': 'Fraction of grass',
        'irr': 'Fraction of irrigated grass',
        'watr': 'Fraction of water',
        'H': 'Average building height (m)',
        'W': 'Average street width (m)',
    }


    # Define variable-specific colormaps
    var_cmap_dict = {
        # Surface fractions
        'road': mpl_cm.Greys,
        'conc': mpl_cm.Oranges,
        'roof': mpl_cm.Reds,
        'Veg': mpl_cm.Greens,
        'dry': mpl_cm.Greens,
        'irr': mpl_cm.Blues,
        'watr': mpl_cm.PuBu,
        'H': mpl_cm.inferno,
        'W': mpl_cm.plasma,
    }

    # --- Plot setup
    fig, axes = plt.subplots(3, 3, figsize=(10, 15))
    axes = axes.flatten()

    for i, var in enumerate(surface_vars):
        ax = axes[i]
        cmap = var_cmap_dict.get(var, mpl_cm.viridis)
        vmin = gdf_merged[var].min()
        vmax = gdf_merged[var].max()
        norm = mpl_colors.Normalize(vmin=vmin, vmax=vmax)

        # Plot the hexes
        gdf_merged.plot(
            column=var,
            cmap=cmap,
            linewidth=0.2,
            edgecolor='black',
            ax=ax,
            norm=norm
        )

        ax.set_title(pretty_names[var], fontsize=11)
        ax.set_axis_off()

        # Add horizontal colorbar under the subplot
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
        sm._A = []
        cbar = fig.colorbar(
            sm,
            ax=ax,
            orientation='horizontal',
            fraction=0.035,
            pad=0.04,
            shrink=0.7,
            aspect=20
        )
        # cbar.set_label(pretty_names[var], fontsize=9)
        cbar.set_label("")
        cbar.ax.tick_params(labelsize=8)

    # Hide any unused axes if fewer than 9
    for j in range(len(surface_vars), len(axes)):
        axes[j].axis("off")

    plt.tight_layout()

    plt.savefig(figname, dpi=300, bbox_inches='tight', pad_inches=0.15)
    print("File saved:", os.path.exists(figname))

    plt.show()


In [ ]:
def plot_heatwave_characteristics(MODEL_INPUT_FOLDER, figname):

    import os
    import pandas as pd
    import matplotlib.pyplot as plt

    met_file = os.path.join(MODEL_INPUT_FOLDER, "MET", "target_forcing.csv")

    # Read the file using whitespace delimiter and header in first row
    df = pd.read_csv(met_file, index_col="datetime", parse_dates=True)

    # plot
    fig, axes = plt.subplots(5, 1, figsize=(12, 10), sharex=True)

    # Plot Kdown
    ax_i = 0
    axes[ax_i].plot(df.index, df['Kd'], color='#FFBF00', label='Kdown')
    axes[ax_i].set_ylabel("SW Down (W/m²)")
    axes[ax_i].grid(True)
    axes[ax_i].legend()
    # axes[ax_i].axvspan(df_selection.index[0], df_selection.index[-1], color='orange', alpha=0.2)

    # Plot Tair
    ax_i = 1
    axes[ax_i].plot(df.index, df['Ta'], color='firebrick', label='Tair')
    axes[ax_i].set_ylabel("Temperature (°C)")
    axes[ax_i].grid(True)
    axes[ax_i].legend()
    # axes[ax_i].axvspan(df_selection.index[0], df_selection.index[-1], color='orange', alpha=0.2)

    # Plot RH
    ax_i = 2
    axes[ax_i].plot(df.index, df['RH'], color='navy', label='RH')
    axes[ax_i].set_ylabel("Relative Humidity (%)")
    axes[ax_i].grid(True)
    axes[ax_i].legend()
    # axes[ax_i].axvspan(df_selection.index[0], df_selection.index[-1], color='orange', alpha=0.2)

    # Plot U
    ax_i = 3
    axes[ax_i].plot(df.index, df['WS'], color='olive', label='U')
    axes[ax_i].set_ylabel("Wind Speed (m/s)")
    axes[ax_i].set_xlabel("Time")
    axes[ax_i].grid(True)
    axes[ax_i].legend()
    # axes[ax_i].axvspan(df_selection.index[0], df_selection.index[-1], color='orange', alpha=0.2)

    # Plot wdir
    ax_i = 4
    axes[ax_i].plot(df.index, df['WD'], color='0.2', label='WDir')
    axes[ax_i].set_ylabel("Wind Direction (° from N)")
    axes[ax_i].set_xlabel("Time")
    axes[ax_i].grid(True)
    axes[ax_i].legend()
    # axes[ax_i].axvspan(df_selection.index[0], df_selection.index[-1], color='orange', alpha=0.2)

    plt.tight_layout()

    plt.savefig(figname, dpi=300, bbox_inches='tight', pad_inches=0.15)
    print("File saved:", os.path.exists(figname))

    plt.show()

In [ ]:
def update_target_config(
    city,
    cityInfo,
    scenario_name,
    MODEL_CONFIG_INITIAL,
    MODEL_CONFIG_UPDATED,
    MODEL_PARAM_INITIAL,
    MODEL_INPUT_FOLDER,
    MODEL_OUTPUT_FOLDER,
):

    # Read initial config file, and adjust to site specific values
    import os
    import configparser

    # Read the file
    config_path = MODEL_CONFIG_INITIAL
    config = configparser.RawConfigParser()
    config.optionxform = str  # preserve case
    config.read(config_path)

    # Step 2: Modify parameters in the DEFAULT section
    # Adjust relevant values
    config['DEFAULT']['para_json_path'] = MODEL_PARAM_INITIAL # Values will be updated later on
    config['DEFAULT']['work_dir'] = MODEL_OUTPUT_FOLDER
    config['DEFAULT']['site_name'] = city
    config['DEFAULT']['run_name'] = 'output'  # Used as filename for output
    config['DEFAULT']['inpt_met_file'] = os.path.join(MODEL_INPUT_FOLDER, "MET", "target_forcing.csv")
    config['DEFAULT']['inpt_lc_file'] = os.path.join(MODEL_INPUT_FOLDER, "LC", "lc_target.csv")
    # config['DEFAULT']['date_fmt'] = '%d/%m/%Y %H:%M'  # format of datetime in input met files
    config['DEFAULT']['timestep'] = '60'
    config['DEFAULT']['include roofs'] = 'N'
    config['DEFAULT']['lat'] = str(get_city_info(city)['lat'])
    config['DEFAULT']['domainDim'] = str(get_city_info(city)['domainDim'])
    config['DEFAULT']['date1a'] = get_city_info(city)['date1a']
    config['DEFAULT']['date1'] = get_city_info(city)['date1']
    config['DEFAULT']['date2'] = get_city_info(city)['date2']

    # Check content of config.ini
    for key, value in config['DEFAULT'].items():
        print(f"{key} = {value}")
    print()

    # Step 3: Save to a new location
    with open(MODEL_CONFIG_UPDATED, 'w') as configfile:
        config.write(configfile)
    print(f"Updated config.ini saved to:\n {MODEL_CONFIG_UPDATED}")

    return config

In [ ]:
def change_param_values(tar, city):

    # For Davao, do not change concrete to bare soil!
    if city == 'Davao':
        print("> No parameter changes needed for Davao...")

    elif city == 'Timbuktu':

        # Check default parameters for "concrete" (that represents bare soil in TARGET)
        print("> Values before change ...")
        for param in ['alb', 'emis', 'C', 'K', 'LUMPS1', 'alphapm', 'beta']:
            print(f"{param}: {tar.parameters[param]['conc']}")

        # consider 'conc' as bare soil, set values accordingly
        tar.parameters['alb']['conc'] = 0.38 # From Modis - see GEE script
        tar.parameters['emis']['conc'] = 0.478 # From Modis - see GEE script
        tar.parameters['C']['conc'] = 1168000.0 # From SOILGRIDS - see GEE script
        tar.parameters['K']['conc'] = 0.00000224 # From SOILGRIDS - see GEE script

        # What about the LUMPS parameters?
        tar.parameters['LUMPS1']['conc'] = tar.parameters['LUMPS1']['dry'] # Surface Energy Budget Coefficients
        tar.parameters['alphapm']['conc'] = tar.parameters['alphapm']['dry'] # Surface Moisture Availability Coefficient
        tar.parameters['beta']['conc'] = tar.parameters['beta']['dry'] # Empirical Stability or Radiation Exchange Coefficient

        print("\n> Values after change ...")
        for param in ['alb', 'emis', 'C', 'K', 'LUMPS1', 'alphapm', 'beta']:
            print(f"{param}: {tar.parameters[param]['conc']}")


In [ ]:
def target_npy_to_df(fn_file, variable):

    """ Convert TARGET pickle output to Dataframe """
    import numpy as np
    import pandas as pd

    print(f"> Reading TARGET .npy output ...")
    result = np.load(fn_file, allow_pickle=True)

    # Get the FIDs
    fids = result[0, :, 0]['ID'].flatten()

    # Initialize df output all
    all_dfs = []

    for i, fid in enumerate(fids):
        tmp = result[:, i, 0]
        df_tmp = pd.DataFrame(tmp)
        df_tmp = df_tmp.set_index(['date'], drop=True)[variable]
        df_tmp.name = fid
        df_tmp = pd.DataFrame(df_tmp)
        all_dfs.append(df_tmp)

    df_out = pd.concat(all_dfs, axis=1)
    df_out.index = pd.to_datetime(df_out.index)

    return df_out


def load_target_output(model_output_folder, variable):

    """Helper function"""

    import numpy as np
    import pandas as pd
    import os

    # First, load the results, by selecting one variable of interest
    fn_file = os.path.join(model_output_folder, "output.npy")
    df_out = target_npy_to_df(fn_file, variable)

    return df_out


def process_target_output(model_output_folder, variable):

    """Helper function"""

    import numpy as np
    import pandas as pd
    import os

    # First, load the results, by selecting one variable of interest
    df_out = load_target_output(model_output_folder, variable)

    # Compute daily stats
    variable_min = df_out.resample('D').min().mean().round(1)
    variable_mean = df_out.resample('D').mean().mean().round(1)
    variable_max = df_out.resample('D').max().mean().round(1)

    df_stats = pd.concat([variable_min, variable_mean, variable_max], axis=1)
    df_stats.columns = [f"{variable}_min", f"{variable}_mean", f"{variable}_max"]

    return df_stats



In [ ]:
def plot_daily_stats_on_interactive_map(MODEL_OUTPUT_FOLDER, MODEL_INPUT_FOLDER, variable):

    import geopandas as gpd
    import folium
    import matplotlib.cm as mpl_cm
    import matplotlib.colors as mpl_colors
    import branca.colormap as cm
    import numpy as np
    import pandas as pd
    from IPython.display import display, clear_output
    import ipywidgets as widgets
    import os

    # First, load the results, by selecting one variable of interest
    df_stats = process_target_output(MODEL_OUTPUT_FOLDER, variable)

    # Merge with hex grid
        # Read the hexagons
    gdf_hex = gpd.read_file(
        os.path.join(os.path.dirname(MODEL_INPUT_FOLDER), "roi_hex_utm.geojson")
    ).set_index('hex_id')
    gdf_hex.index.name = 'FID'
    gdf_merged = gdf_hex.join(df_stats)

    # Ensure WGS84
    gdf_merged = gdf_merged.to_crs(epsg=4326)

    # # Define variable-specific colormaps
    # var_cmap_dict = {
    #     # Temperature statistics
    #     f"{variable}_min": mpl_cm.hot_r,
    #     f"{variable}_mean": mpl_cm.hot_r,
    #     f"{variable}_max": mpl_cm.hot_r,
    # }

    # Create dropdown menu
    dropdown = widgets.Dropdown(
        options=df_stats.columns,
        value= f"{variable}_mean",
        description='Variable:',
        layout=widgets.Layout(width='50%')
    )

    # Output widget for the map
    out = widgets.Output()

    # Map update function
    def update_map(change=None):
        variable = dropdown.value
        # cmap = var_cmap_dict.get(variable, mpl_cm.viridis)
        cmap = mpl_cm.hot_r

        vmin = gdf_merged[variable].min()
        vmax = gdf_merged[variable].max()

        scalar_map = cm.StepColormap(
            colors=[mpl_colors.to_hex(cmap(i)) for i in np.linspace(0, 1, 256)],
            vmin=vmin, vmax=vmax,
            caption=variable
        )

        center = [
            gdf_merged.geometry.centroid.y.mean(),
            gdf_merged.geometry.centroid.x.mean()
        ]

        m = folium.Map(
            location=center,
            zoom_start=14,
            # tiles='cartodbpositron',
            tiles="https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}",
            attr='Google',
            name='Google Satellite',
            width='90%',
            height='750px'
        )

            # Add hex outlines
        folium.GeoJson(
            gdf_merged,
            name="Hex borders",
            style_function=lambda x: {'fillOpacity': 0, 'color': 'blue', 'weight': 1}
        ).add_to(m)

        def style_function(feature):
            val = feature['properties'].get(variable)
            return {
                'fillOpacity': 0.7,
                'weight': 0.2,
                'color': 'black',
                'fillColor': scalar_map(val) if val is not None else '#999999'
            }

        folium.GeoJson(
            gdf_merged.to_json(),
            style_function=style_function,
            tooltip=folium.GeoJsonTooltip(
                fields=[f"{variable}"],
                aliases=[f"{variable}: "]
            )
        ).add_to(m)

        scalar_map.add_to(m)

        with out:
            clear_output(wait=True)
            display(m)

    # Connect dropdown to map update
    dropdown.observe(update_map, names='value')

    # Initial display
    update_map()

    # Display dropdown and map
    display(widgets.VBox([dropdown, out]))


In [ ]:
def plot_daily_stats_on_static_map(MODEL_OUTPUT_FOLDER, MODEL_INPUT_FOLDER, variable, figname):

    import geopandas as gpd
    import matplotlib.pyplot as plt
    import matplotlib.cm as mpl_cm
    import matplotlib.colors as mpl_colors
    import numpy as np
    import pandas as pd
    import os

    # First, load the results, by selecting one variable of interest
    df_stats = process_target_output(MODEL_OUTPUT_FOLDER, variable)

    # Merge with hex grid
        # Read the hexagons
    gdf_hex = gpd.read_file(
        os.path.join(os.path.dirname(MODEL_INPUT_FOLDER), "roi_hex_utm.geojson")
    ).set_index('hex_id')
    gdf_hex.index.name = 'FID'
    gdf_merged = gdf_hex.join(df_stats)

    # Ensure WGS84
    gdf_merged = gdf_merged.to_crs(epsg=4326)

    fig, axes = plt.subplots(1, 3, figsize=(14, 6), constrained_layout=True)
    # plt.subplots_adjust(wspace=0.01)  # reduce horizontal spacing between plots

    for ax, var in zip(axes, df_stats.columns):
        cmap =  mpl_cm.hot_r
        vmin = gdf_merged[var].min()
        vmax = gdf_merged[var].max()
        norm = mpl_colors.Normalize(vmin=vmin, vmax=vmax)

        # Plot
        gdf_merged.plot(
            column=var,
            cmap=cmap,
            linewidth=0.2,
            edgecolor='black',
            ax=ax,
            norm=norm
        )

        ax.set_title(f"{var}", fontsize=14)
        ax.set_axis_off()

        # Colorbar below each subplot
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
        sm._A = []  # dummy array for colorbar
        cbar = fig.colorbar(sm, ax=ax, orientation='horizontal',
                            fraction=0.05, shrink=0.6, pad=0.02)
        cbar.set_label(f"{var} (°C)", fontsize=10)

    plt.savefig(figname, dpi=300, bbox_inches='tight', pad_inches=0.15)
    print("File saved:", os.path.exists(figname))

    plt.show()

In [ ]:
def scatter_variable_vs_surface(MODEL_OUTPUT_FOLDER, MODEL_INPUT_FOLDER, variable, variable_prop, figname):

    import pandas as pd
    import geopandas as gpd
    import matplotlib.pyplot as plt
    import numpy as np
    import os

    # First, load the results, by selecting one variable of interest
    df_stats = process_target_output(MODEL_OUTPUT_FOLDER, variable)

    # Merge with hex grid
    gdf_hex = gpd.read_file(
          os.path.join(os.path.dirname(MODEL_INPUT_FOLDER), "roi_hex_utm.geojson")
      ).set_index('hex_id')
    gdf_hex.index.name = 'FID'
    gdf_merged = gdf_hex.join(df_stats)

    # Read the fractions
    df_fractions = pd.read_csv(
        os.path.join(MODEL_INPUT_FOLDER, "LC", "lc_target.csv"),
        index_col="FID"
    )

    # Merge with hexagons
    # gdf_merged = gdf_hex.merge(df_fractions, left_on='hex_id', right_index=True, how='left')
    gdf_merged = gdf_merged.join(df_fractions)

    surface_variables = ['road', 'conc', 'roof', 'Veg', 'dry', 'irr', 'watr', 'H', 'W']

    # Plot scatter plots for each surface variable
    plt.figure(figsize=(15, 10)) # Adjust the overall figure size

    for i, var in enumerate(surface_variables):
        plt.subplot(3, 3, i + 1) # Create subplots (3 rows, 3 columns)
        plt.scatter(gdf_merged[var], gdf_merged[f'{variable}_{variable_prop}'])
        plt.title(f'{variable}_{variable_prop} vs {var}')
        plt.xlabel(var)
        plt.ylabel(f'{variable}_{variable_prop}')
        plt.grid(True)

    plt.tight_layout() # Adjust layout to prevent overlapping

    plt.savefig(figname, dpi=300, bbox_inches='tight', pad_inches=0.15)
    print("File saved:", os.path.exists(figname))

    plt.show()

In [ ]:
def plot_timesereries_hex_of_interest(MODEL_OUTPUT_FOLDER, variable, hex_ids, figname):

    import geopandas as gpd
    import matplotlib.pyplot as plt
    import matplotlib.cm as mpl_cm
    import matplotlib.colors as mpl_colors
    import numpy as np
    import pandas as pd
    import os

    # First, load the results, by selecting one variable of interest
    fn_file = os.path.join(MODEL_OUTPUT_FOLDER, "output.npy")
    df_out = target_npy_to_df(fn_file, variable)

    # Create the plot
    plt.figure(figsize=(12, 6))
    for hex_id in hex_ids:
      plt.plot(df_out.index, df_out[hex_id], label=f'Hex ID: {hex_id}')

    plt.xlabel("Time")
    plt.ylabel(variable)
    plt.title(f"Timeseries of {variable} for selected Hex IDs")
    plt.legend()
    plt.grid(True)

    plt.savefig(figname, dpi=300, bbox_inches='tight', pad_inches=0.15)
    print("File saved:", os.path.exists(figname))

    plt.show()

In [ ]:
def make_interactive_map_roi_compare(MODEL_INPUT_FOLDER_BASE, MODEL_INPUT_FOLDER_SCEN, variable='Veg'):

    import os
    import geopandas as gpd
    import folium
    import matplotlib.cm as mpl_cm
    import matplotlib.colors as mpl_colors
    import branca.colormap as cm
    import numpy as np
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    import pandas as pd

    # Read the hexagons
    gdf_hex = gpd.read_file(
        os.path.join(os.path.dirname(MODEL_INPUT_FOLDER_BASE), "roi_hex_utm.geojson")
    ).set_index('hex_id')

    # Read both fractions
    def read_fractions(folder):
        df = pd.read_csv(os.path.join(folder, "LC", "lc_target.csv"), index_col="FID")
        return df[[variable]].rename(columns={variable: f"{variable}_{os.path.basename(folder)}"}), f"{variable}_{os.path.basename(folder)}"

    df_base, col_base = read_fractions(MODEL_INPUT_FOLDER_BASE)
    df_scen, col_scen = read_fractions(MODEL_INPUT_FOLDER_SCEN)

    # Merge with hex grid
    gdf_merged = gdf_hex.join(df_base).join(df_scen)

    # Compute difference
    gdf_merged["diff"] = gdf_merged[col_scen] - gdf_merged[col_base]

    gdf_merged["hex_id"] = gdf_merged.index  # restore as column

    # Ensure WGS84
    gdf_merged = gdf_merged.to_crs(epsg=4326)

    # Variables to plot
    vars_to_plot = [col_base, col_scen, "diff"]

    # Create dropdown menu
    dropdown = widgets.Dropdown(
        options=vars_to_plot,
        value=col_base,
        description='Variable:',
        layout=widgets.Layout(width='50%')
    )

    # Output widget for the map
    out = widgets.Output()

    # Map update function
    def update_map(change=None):
        variable = dropdown.value
        if variable == "diff":
            cmap = mpl_cm.PuOr
        else:
            cmap = mpl_cm.Greys

        vmin = gdf_merged[variable].min()
        vmax = gdf_merged[variable].max()

        scalar_map = cm.StepColormap(
            colors=[mpl_colors.to_hex(cmap(i)) for i in np.linspace(0, 1, 256)],
            vmin=vmin, vmax=vmax,
            caption=variable
        )

        center = [
            gdf_merged.geometry.centroid.y.mean(),
            gdf_merged.geometry.centroid.x.mean()
        ]

        m = folium.Map(
            location=center,
            zoom_start=14,
            # tiles='cartodbpositron',
            tiles="https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}",
            attr='Google',
            name='Google Satellite',
            width='90%',
            height='750px'
        )

        def style_function(feature):
            val = feature['properties'].get(variable)
            return {
                'fillOpacity': 0.7,
                'weight': 0.2,
                'color': 'black',
                'fillColor': scalar_map(val) if val is not None else '#999999'
            }

        folium.GeoJson(
            gdf_merged.to_json(),
            style_function=style_function,
            tooltip=folium.GeoJsonTooltip(
                fields=["hex_id", variable],
                aliases=["Hex ID: ", variable + ": "],
                localize=True
            )
        ).add_to(m)

        scalar_map.add_to(m)

        with out:
            clear_output(wait=True)
            display(m)

    # Connect dropdown to map update
    dropdown.observe(update_map, names='value')

    # Initial display
    update_map()

    # Display dropdown and map
    display(widgets.VBox([dropdown, out]))

In [ ]:
def plot_daily_stats_on_static_map_scenarios(MODEL_OUTPUT_FOLDER_BASE, MODEL_OUTPUT_FOLDER_SCEN, MODEL_INPUT_FOLDER, variable, figname):

    import geopandas as gpd
    import matplotlib.pyplot as plt
    import matplotlib.cm as mpl_cm
    import matplotlib.colors as mpl_colors
    import numpy as np
    import pandas as pd
    import os

    # Load both outputs
    df_stats_base = process_target_output(MODEL_OUTPUT_FOLDER_BASE, variable)
    df_stats_scen = process_target_output(MODEL_OUTPUT_FOLDER_SCEN, variable)
    df_stats_diff = df_stats_scen - df_stats_base

    # Merge with hex grid
        # Read the hexagons
    gdf_hex = gpd.read_file(
        os.path.join(os.path.dirname(MODEL_INPUT_FOLDER), "roi_hex_utm.geojson")
    ).set_index('hex_id')
    gdf_hex.index.name = 'FID'

    # Merge with hex grid
    gdf_base = gdf_hex.join(df_stats_base)
    gdf_scen = gdf_hex.join(df_stats_scen)
    gdf_diff = gdf_hex.join(df_stats_diff)

    # Ensure WGS84
    gdf_base = gdf_base.to_crs(epsg=4326)
    gdf_scen = gdf_scen.to_crs(epsg=4326)
    gdf_diff = gdf_diff.to_crs(epsg=4326)

    # Plot
    fig, axes = plt.subplots(3, 3, figsize=(10, 12), constrained_layout=True)

    for i, (gdf, label, cmap, center_zero) in enumerate([
        (gdf_base, "Baseline", mpl_cm.hot_r, False),
        (gdf_scen, "Scenario", mpl_cm.hot_r, False),
        (gdf_diff, "Scenario-Baseline", mpl_cm.coolwarm, True)
    ]):
        for j, var in enumerate(df_stats_base.columns):
            ax = axes[i, j]

            if center_zero:
                vmax = np.abs(gdf[var]).max()
                vmin = -vmax
                norm = mpl_colors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)
            else:
                vmin = gdf[var].min()
                vmax = gdf[var].max()
                norm = mpl_colors.Normalize(vmin=vmin, vmax=vmax)

            gdf.plot(
                column=var,
                cmap=cmap,
                linewidth=0.2,
                edgecolor='black',
                ax=ax,
                norm=norm
            )

            ax.set_title(f"{label} - {var}", fontsize=12)
            ax.set_axis_off()

            sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
            sm._A = []
            cbar = fig.colorbar(sm, ax=ax, orientation='horizontal',
                                fraction=0.05, shrink=0.6, pad=0.02)
            cbar.set_label(f"{var} (°C)", fontsize=9)

    plt.savefig(figname, dpi=300, bbox_inches='tight', pad_inches=0.15)
    print("File saved:", os.path.exists(figname))
    plt.show()

In [ ]:
def plot_timesereries_hex_of_interest_scen(MODEL_OUTPUT_FOLDER_BASE, MODEL_OUTPUT_FOLDER_SCEN, variable, hex_ids, figname):

    import geopandas as gpd
    import matplotlib.pyplot as plt
    import matplotlib.cm as mpl_cm
    import matplotlib.colors as mpl_colors
    import numpy as np
    import pandas as pd
    import os

    # First, load the results, by selecting one variable of interest
    fn_file_base = os.path.join(MODEL_OUTPUT_FOLDER_BASE, "output.npy")
    df_out_base = target_npy_to_df(fn_file_base, variable)

    fn_file_scen = os.path.join(MODEL_OUTPUT_FOLDER_SCEN, "output.npy")
    df_out_scen = target_npy_to_df(fn_file_scen, variable)

    poi_colors = ['#e41a1c', '#377eb8', '#4daf4a', '#984ea3', '#ff7f00']

    # Create the plot
    plt.figure(figsize=(12, 6))
    for hi, hex_id in enumerate(hex_ids):
      plt.plot(df_out_base.index, df_out_base[hex_id], color=poi_colors[hi], label=f'Hex ID: {hex_id} (BASE)')
      plt.plot(df_out_scen.index, df_out_scen[hex_id], color=poi_colors[hi], label=f'Hex ID: {hex_id} (SCEN)', ls=':')

    plt.xlabel("Time")
    plt.ylabel(variable)
    plt.title(f"Timeseries of {variable} for selected Hex IDs")
    plt.legend()
    plt.grid(True)

    plt.savefig(figname, dpi=300, bbox_inches='tight', pad_inches=0.15)
    print("File saved:", os.path.exists(figname))

    plt.show()